Lambda School Data Science

*Unit 4, Sprint 1, Module 3*

---

# Document Classification (Assignment)

This notebook is for you to practice skills during lecture.

Today's guided module project and assignment will be different. You already know how to do classification. You ready know how to extract features from documents. So? That means you're ready to combine and practice those skills in a kaggle competition. We we will open with a five minute sprint explaining the competition, and then give you 25 minutes to work. After those twenty five minutes are up, I will give a 5-minute demo an NLP technique that will help you with document classification (*and **maybe** the competition*).

Today's all about having fun and practicing your skills.

## Sections
* <a href="#p1">Part 1</a>: Text Feature Extraction & Classification Pipelines
* <a href="#p2">Part 2</a>: Latent Semantic Indexing
* <a href="#p3">Part 3</a>: Word Embeddings with Spacy
* <a href="#p4">Part 4</a>: Post Lecture Assignment

# Text Feature Extraction & Classification Pipelines (Learn)
<a id="p1"></a>

## Follow Along 

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model (try using the pipe method I just demoed)

### Load Competition Data

In [27]:
import pandas as pd

# You may need to change the path
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')

In [28]:
pd.set_option('max_colwidth', 0)
train.head(2)

,id,description,ratingCategory
0,1321,"\nSometimes, when whisky is batched, a few leftover barrels are returned to the warehouse. Canadian Club recently pulled and vatted several of these from the 1970s. Acetone, Granny Smith apples, and fresh-cut white cedar showcase this long age. Complex and spicy, yet reserved, this dram is ripe with strawberries, canned pears, cloves, pepper, and faint flowers, then slightly pulling oak tannins. Distinct, elegant, and remarkably vibrant, this ancient Canadian Club is anything but tired. (Australia only) A$133",1
1,3861,"\nAn uncommon exclusive bottling of a 6 year old cask strength malt. Light gold in color, the nose is vegetal, more peat bog than peat smoke, with an undercurrent of pastry cream and rose. It’s an odd combination of aromas. The entry is flavorful and inviting with smoked pineapple, clove, and rose. Peak smoke arrives in full force in the mid-palate, which drops the sweet and becomes spicy. The finish is mostly smoke, but with a pleasant minty coolness. (Wyoming only)",0


In [29]:
train.shape

(4087, 3)

In [30]:
test.shape

(1022, 2)

In [31]:
test.head(2)

,id,description
0,3461,"\nStyle: Speyside single malt scotch Color: Walnut Aroma: Richly sherried and thick, with notes of nuts and toffee. Wood resins contribute spice and variety. Fruitcake at Christmas. Palate: Thick, chewy in texture, and quite ripe. Again the fruitcake. Very deep and mature with some underlying maltiness. Dry, spicy, oak notes fight off all that sherry and add balance and complexity. Long, soothing finish. \r\n"
1,2604,"\nVery bright and lively, with a nice balance of flavors. Zesty fruit (lemon, peach, ripe pineapple, golden raisin) on a bed of layered sweetness (creamy vanilla, light honey, lightly toasted marshmallow, and a hint of coconut). Gently dry, delicately spicy, dried citrus finish. Light enough and with enough zing to enjoy before dinner, but it should stand up well enough after dinner, too. This is a nice whisky, but it shows a lighter, more elegant side of Glenrothes. It doesn’t express the rich, opulent notes often shown in bottlings like the 1972 Vintage, for example."


### Define Pipeline Components

In [32]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer

In [33]:
vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2))
rfc = RandomForestClassifier()

pipe = Pipeline([
    ('vect', vect), 
    ('rfc', rfc)
])

### Define Your Search Space
You're looking for both the best hyperparameters of your vectorizer and your classification model. 

In [34]:
parameters = {
    'vect__max_df': (0.6, 0.9),
    'vect__min_df': (.02, .05),
    'vect__max_features': (5000, None),
    'rfc__n_estimators': (100, 250),
    'rfc__max_depth': (15, None)
}

parameters = {
    'vect__min_df': (0.01, 0.02),
    'vect__max_df': (0.5, 0.6),
    'vect__max_features': (500, None, 1000),
    'rfc__max_depth':(None, 5),
    'rfc__n_estimators': (250,500)
}

grid_search = GridSearchCV(pipe, parameters, cv=3, n_jobs=4, verbose=1)
grid_search.fit(train['description'], train['ratingCategory'])

Fitting 3 folds for each of 48 candidates, totalling 144 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:  1.1min
[Parallel(n_jobs=4)]: Done 144 out of 144 | elapsed:  3.0min finished


GridSearchCV(cv=3, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        TfidfVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.float64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 2),
                                                        no

In [35]:
grid_search.best_params_

{'rfc__max_depth': None,
 'rfc__n_estimators': 250,
 'vect__max_df': 0.6,
 'vect__max_features': None,
 'vect__min_df': 0.01}

In [36]:
grid_search.best_score_

0.743089245204623

In [37]:
# from sklearn.metrics import accuracy_score

# # Evaluate on test data
# y_test = grid_search.predict(test['description'])
# # accuracy_score((test['description']).target, y_test)

### Make a Submission File
*Note:* In a typical Kaggle competition, you are only allowed two submissions a day, so you only submit if you feel you cannot achieve higher test accuracy. For this competition the max daily submissions are capped at **20**. Submit for each demo and for your assignment. 

In [38]:
# Predictions on test sample
pred = grid_search.predict(test['description'])

In [39]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [40]:
# Make Sure the Category is an Integer
submission.head()

,id,ratingCategory
0,3461,1
1,2604,1
2,3341,1
3,3764,1
4,2306,1


In [41]:
subNumber = 0

In [42]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'./dspt4-which-whiskey/submission{subNumber}.csv', index=False)
subNumber += 1

FileNotFoundError: [Errno 2] No such file or directory: './dspt4-which-whiskey/submission0.csv'

## Challenge

You're trying to achieve a minimum of 70% Accuracy on your model.

## Latent Semantic Indexing (Learn)
<a id="p2"></a>

## Follow Along
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (lsi) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
4. Make a submission to Kaggle 


### Define Pipeline Components

In [ ]:
params = {
    'vect__max_df':(0.6, 0.95),
    'svd__n_components': (100, 250),
    'clf__n_estimators': (50, 100)
}

In [43]:
import scipy.stats as stats
from sklearn.model_selection import RandomizedSearchCV
from sklearn.decomposition import TruncatedSVD
import xgboost as xgb
# Create Pipeline Components

vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2))
rfc = RandomForestClassifier()

svd = TruncatedSVD(n_components=100, # Just here for demo. 
                   algorithm='randomized',
                   n_iter=10)
# Instantiate vectorizer object
tfidf = TfidfVectorizer(stop_words = 'english',
                         #tokenizer = tokenize,
                         ngram_range = (1,2),
                         min_df = 0.25, 
                         max_df = 0.8,
                         max_features = 5000
                       )


In [44]:
pipe = Pipeline([
    ('vect', vect),      # TF-IDF Vectorizer
    ('svd', svd),        # Truncated SVD Dimensionality Reduction
    ('clf', rfc),         # RandomForest Classifier
])

### Define Your Search Space
You're looking for both the best hyperparameters of your vectorizer and your classification model. 

In [45]:
# parameters = {
#     'lsi__svd__n_components': [10,100,250],
#     'vect__max_df': (0.75, 1.0),
#     'clf__max_depth':(5,10,15,20)
# }

grid_search = GridSearchCV(pipe,params, cv=5, n_jobs=4, verbose=1)
grid_search.fit(train['description'], train['ratingCategory'])

NameError: name 'params' is not defined

### Make a Submission File

In [ ]:
# Predictions on test sample
pred = grid_search.predict(test['description'])

In [ ]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [ ]:
# Make Sure the Category is an Integer
submission.head()

In [ ]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'./data/submission{subNumber}.csv', index=False)
subNumber += 1

## Challenge

Continue to apply Latent Semantic Indexing (LSI) to various datasets. 

# Word Embeddings with Spacy (Learn)
<a id="p3"></a>

## Follow Along

In [57]:
# Apply to your Dataset

import sys
!{sys.executable} -m pip install xgboost
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import GradientBoostingClassifier

from scipy.stats import randint
from xgboost import XGBClassifier

# param_dist = {
    
#     'max_depth' : (1,150,300),
#     'min_samples_leaf': (1, 0),
#     'n_estimators' : (10,200,400),
#     'bootstrap': (True, False),
#     'max_features': ('auto', 'sqrt'),
#     'min_samples_leaf': [1, 2, 4],
#     'min_samples_split': [2, 5, 10],
#}
param_dist = {
    'eta' : (0.1, 0.3),
    'min_child_weight' : (1,2,4),
    'max_depth' : (1,4,7,9),
    'alpha' : (0,1,3),

}
params = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5]
        }
# param_dist = {
#     'bootstrap': [True, False],
#     'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
#     'max_features': ['auto', 'sqrt'],
#     'min_samples_leaf': [1, 2, 4],
#     'min_samples_split': [2, 5, 10],
#     'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000
# }

In [47]:
# Continue Word Embedding Work Here
import spacy
nlp = spacy.load("en_core_web_lg")


In [62]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=3, shuffle = True, random_state = 1001)

In [49]:
def get_word_vectors(docs):
    return [nlp(doc).vector for doc in docs]

In [50]:
X = get_word_vectors(train['description'])
len(X) == len(train['description'])

True

In [71]:
X_df=pd.DataFrame(X)

In [72]:
X_df.head()

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,-0.046755,0.217486,-0.139249,-0.086381,0.108063,0.082181,-0.069926,-0.078897,-0.046305,1.675343,...,-0.163038,0.011581,-0.037443,0.014240,-0.143216,0.018097,0.000748,-0.209037,-0.072346,0.042847
1,-0.019244,0.206962,-0.056247,-0.029087,0.079248,-0.001398,-0.024880,-0.057035,-0.025045,1.641834,...,-0.161042,0.091476,-0.045018,-0.110397,-0.127780,0.029475,0.054958,-0.138565,-0.067064,0.065419
2,0.046398,0.189816,-0.111451,-0.031625,0.081704,-0.025362,-0.026995,-0.062688,-0.021074,1.624273,...,-0.109123,0.022506,-0.044898,-0.077540,-0.114823,0.008062,0.054627,-0.146851,-0.135264,0.072096
3,-0.064541,0.224933,-0.091545,-0.078598,0.032475,0.032356,-0.103998,-0.050203,-0.022012,1.528916,...,-0.110060,0.047583,-0.052992,-0.065310,-0.171562,0.033367,0.090011,-0.240344,-0.095537,0.113190
4,-0.092036,0.261761,-0.188370,-0.070761,0.026548,0.098266,-0.133690,-0.031325,-0.051093,1.566256,...,-0.128262,0.054398,-0.086081,-0.126515,-0.222649,0.035877,0.110670,-0.330467,-0.044748,0.109515


In [58]:
xgb = XGBClassifier(learning_rate=0.02, n_estimators=600, objective='binary:logistic',
                    silent=True, nthread=1)

In [76]:
grid_search = GridSearchCV(xgb,params, cv=5, n_jobs=4, verbose=1)
#GridSearchCV(estimator=xgb, param_grid=params, scoring='roc_auc', n_jobs=4, cv=skf.split(X_df,train['ratingCategory']), verbose=3 )
#


In [ ]:
grid_search.fit(X_df, train['ratingCategory'])

Fitting 5 folds for each of 405 candidates, totalling 2025 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed: 19.5min
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed: 607.5min
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed: 744.1min
[Parallel(n_jobs=4)]: Done 792 tasks      | elapsed: 991.8min


In [ ]:
grid_search.best_params_

In [98]:
X_test = get_word_vectors(test['description'])

In [67]:
X

AttributeError: 'list' object has no attribute 'shape'

In [ ]:
grid_search.best_score_

In [85]:
rfc.fit(X, train['ratingCategory'])

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [86]:
test.head(2)

,id,description
0,3461,"\nStyle: Speyside single malt scotch Color: Walnut Aroma: Richly sherried and thick, with notes of nuts and toffee. Wood resins contribute spice and variety. Fruitcake at Christmas. Palate: Thick, chewy in texture, and quite ripe. Again the fruitcake. Very deep and mature with some underlying maltiness. Dry, spicy, oak notes fight off all that sherry and add balance and complexity. Long, soothing finish. \r\n"
1,2604,"\nVery bright and lively, with a nice balance of flavors. Zesty fruit (lemon, peach, ripe pineapple, golden raisin) on a bed of layered sweetness (creamy vanilla, light honey, lightly toasted marshmallow, and a hint of coconut). Gently dry, delicately spicy, dried citrus finish. Light enough and with enough zing to enjoy before dinner, but it should stand up well enough after dinner, too. This is a nice whisky, but it shows a lighter, more elegant side of Glenrothes. It doesn’t express the rich, opulent notes often shown in bottlings like the 1972 Vintage, for example."


In [100]:
# Evaluate on test data
y_test = grid_search.predict(X_test)
# accuracy_score(test['ratingCategory'], y_test)

### Make a Submission File

In [89]:
# # Predictions on test sample
# pred = rfc.predict(test['description'])

In [134]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':y_test})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [114]:
# Make Sure the Category is an Integer
submission.head()

,id,ratingCategory
0,3461,1
1,2604,1
2,3341,1
3,3764,1
4,2306,1


In [135]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'./data/submission{subNumber}.csv', index=False)
subNumber += 1

## Challenge

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (lsi) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
    - Try to extract word embeddings with Spacy and use those embeddings as your features for a classification model.
4. Make a submission to Kaggle 

# Post Lecture Assignment
<a id="p4"></a>

Your primary assignment this afternoon is to achieve a minimum of 70% accuracy on the Kaggle competition. Once you have achieved 70% accuracy, please work on the following: 

1. Research "Sentiment Analysis". Provide answers in markdown to the following questions: 
    - What is "Sentiment Analysis"? 
    - Is Document Classification different than "Sentiment Analysis"? Provide evidence for your response
    - How do create labeled sentiment data? Are those labels really sentiment?
    - What are common applications of sentiment analysis?
2. Research our why word embeddings worked better for the lecture notebook than on the whiskey competition.
    - This [text classification documentation](https://developers.google.com/machine-learning/guides/text-classification/step-2-5) from Google might be of interest
    - Neural Networks are becoming more popular for document classification. Why is that the case?